In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

In [2]:
import tushare as ts

token = 'c5cba6dd1dc9c4c3578cdd046f54dfc4119fef2aabf7f998ed9a5192'
ts.set_token(token)
pro = ts.pro_api(token)

df=pro.daily(ts_code='000001.SZ', start_date='20030101',
               end_date='20190106')

df.index = pd.to_datetime(df.trade_date)
df.sort_index(ascending=True,inplace = True)  #####

df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,,
2003-01-02,000001.SZ,20030102,10.35,10.35,9.99,10.03,10.49,-0.46,-4.39,64848.85,65629.6488
2003-01-03,000001.SZ,20030103,10.00,10.10,9.91,10.02,10.03,-0.01,-0.10,49594.69,49522.8547
2003-01-06,000001.SZ,20030106,10.03,10.12,9.91,10.06,10.02,0.04,0.40,36553.07,36626.6662
2003-01-07,000001.SZ,20030107,10.06,10.15,10.01,10.04,10.06,-0.02,-0.20,27149.59,27361.4667
2003-01-08,000001.SZ,20030108,10.00,10.74,10.00,10.53,10.04,0.49,4.88,80421.70,83768.8306


In [3]:
print(df.shape)

(3735, 11)


In [18]:
df_s = df[['close','open']]
df_s.loc[:,'gap'] = df_s.close - df_s.open
df_s.loc[:,'mean5'] = df_s.close.rolling(5).mean()
df_s = df_s.drop('open',axis=1).reset_index(drop=True)
df_s = df_s.dropna().reset_index(drop=True)

/home/jameson/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/jameson/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1783: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [19]:
df_s.head()

,close,gap,mean5
0,10.53,0.53,10.136
1,10.69,0.14,10.268
2,10.53,-0.12,10.370
3,10.40,-0.08,10.438
4,11.44,1.03,10.718


In [21]:
print(df_s.shape)

(3731, 3)


In [ ]:
def split_tvs(data,)

In [296]:
class ts_dataset(Dataset):
    
    def __init__(self, data, target_col, categorical_cols,numerical_cols,seq_length,prediction_window):
        self.data = data
        self.target_col = target_col
        self.categorical_cols = categorical_cols
        self.numerical_cols = list(set(data.columns) - set(categorical_cols) - set(target_col))
        self.seq_length = seq_length
        self.prediction_window = prediction_window
        self.preprocessor = None
    
    def __getitem__(self, index):
        X = self.data[index: index+self.seq_length].values
        y = self.data[self.target_col].values[index+self.seq_length : index+self.seq_length+self.prediction_window]
        return X,y
    
    
    def split_tvs(self, X ,y, validation_size, test_size,X_mean,X_std,y_mean,y_std):
        '''split my dataset into training_set, validation_set and prediction_set'''
        
        def split_single(X,y,start,end):
            features, target = [],[]
            
            for i in range(start,end):
                features.append(torch.FloatTensor((X[i: i+self.seq_length, :] - X_mean)/X_std).unsqueeze(0))
                target.append(torch.FloatTensor((y[i + self.seq_length: i+self.seq_length+self.prediction_window]) - y_mean/y_std).unsqueeze(0))
                
            features_var = torch.cat(features)
            target_var = torch.cat(target)
        
            return (features_var, target_var)
            
        train_X, train_y = split_single(X, y, 
                                        0,
                                       len(self.data)-self.seq_length- prediction_size-validation_size-self.prediction_window)
        val_X, val_y = split_single(X,y, 
                                    len(self.data)-self.seq_length- prediction_size-validation_size-self.prediction_window, 
                                    len(self.data)-self.seq_length- prediction_size-self.prediction_window)
        test_X, test_y = split_single(X, y, 
                                      len(self.data) - self.seq_length - prediction_size -self.prediction_window, 
                                      len(self.data) - self.seq_length - self.prediction_window)
        return (train_X,train_y), (val_X,val_y),(test_X, test_y)
    
    
    def preprocess_data(self,validation_size,test_size):
        '''Preprocessing function'''
        #
        X = self.data[numerical_cols + categorical_cols].values
        y = self.data[target_col].values
        
        X_mean = X[:(-validation_size-test_size)].mean(axis=0)
        X_std = X[:(-validation_size-test_size)].std(axis=0)
        
        y_mean = y[:(-validation_size-test_size)].mean()
        y_std = y[:(-validation_size-test_size)].std()
        
        train_set, val_set, test_set = self.split_tvs(X ,y, validation_size, test_size, X_mean,X_std,y_mean,y_std)

        return (train_set, val_set, test_set)
    
    
    def get_loader(self, validation_size,test_size, batch_size: int, drop_last =True):
        
        train_set, val_set, test_set = self.preprocess_data(validation_size, test_size)
        
        train_iter = DataLoader(TensorDataset(*train_set), batch_size = batch_size, shuffle = True, drop_last= drop_last)
        val_iter = DataLoader(TensorDataset(*val_set), batch_size = batch_size, shuffle = True, drop_last= drop_last)
        #test_iter = DataLoader(TensorDataset(*test_set), batch_size = batch_size, shuffle = False, drop_last= drop_last)
        
        return train_iter, val_iter, test_set

In [297]:
target_col = 'close'
categorical_cols = []
numerical_cols = ['close','gap','mean5']
#numerical_cols = ['close']
seq_length = 30
prediction_window = 7

my_dataset = ts_dataset(df_s,target_col,categorical_cols, numerical_cols,seq_length,prediction_window)

In [298]:
train_loader, val_loader, test_loader = my_dataset.get_loader(300,100,128)

In [299]:
iter(train_loader).next()[1].shape

torch.Size([128, 7])

In [300]:
iter(train_loader).next()[0].shape

torch.Size([128, 30, 3])

In [301]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        
        self.lstm = nn.LSTM(
        input_size = 3,
        hidden_size = 128,
        num_layers= 1,
        batch_first = True)
        
        self.out = nn.Sequential(
        nn.Linear(128,1))
        
    def forward(self, x):
        r_out, (h_n, h_c) = self.lstm(x, None)   # 'None' stands for use all zeros as hidden state input
        out = self.out(r_out[:, -7:, :])                     # use the last day as output
        
        return out

In [302]:
LR = 0.0001
EPOCH = 100
EARLY_STOP = True
EARLY_STOP_STEP = 20

In [313]:
rnn = LSTM()

# put model into cuda
if torch.cuda.is_available():
    rnn = rnn.cuda

############optimizer and loss function###############
optimizer = torch.optim.Adam(rnn.parameters(), lr = LR)
loss_func = nn.MSELoss()

best_loss = np.inf
##################################################

if not os.path.exists('weights'):
    os.mkdir('weights')

early_stop_now = 0    

for epoch in range(EPOCH):
    print("--"*60)
    step = 0
    # trainning
    for tx, ty in train_loader:
    
        if torch.cuda.is_available():
            tx = tx.cuda()
            ty = ty.cuda()
            
            ############### 
        output = rnn(tx)
        loss = loss_func(torch.squeeze(output), ty)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 10 == 0:
            print('epoch : %d  ' %epoch, 'step : %d' %step, 'train_loss :  %.4f' % loss.cpu().item())
            
        step += 1
    
    # validation
    with torch.no_grad():
        for tx, ty in val_loader:
            if torch.cuda.is_available():
                tx = tx.cuda()
                ty = ty.cuda()
                
            output = rnn(tx) #    rnn(torch.unsqueeze(tx, dim = 2))
            #unsqueeze: returns a new tensor with a dimension of size one inserted at the specified position
            loss = loss_func(torch.squeeze(output), ty)  #squeeze: retures a tensor with all the dim of input of sieze 1 removed
            
            print('epoch : %d ' % epoch, 'val_loss : %.4f' % loss.cpu().item())
            
            if loss.cpu().item() < best_loss:
                early_stop_now = 0
                best_loss = loss.cpu().item()
                torch.save(rnn.state_dict(), 'weights/rnn.pkl')
                print('new model saved at epoch {} with val_loss {}'.format(epoch, best_loss))
            else:
                early_stop_now += 1
                
    if EARLY_STOP and early_stop_now > EARLY_STOP_STEP:
        print('***************warning: early stop**************')
        break

------------------------------------------------------------------------------------------------------------------------
epoch : 0   step : 0 train_loss :  177.6470
epoch : 0   step : 10 train_loss :  233.8739
epoch : 0   step : 20 train_loss :  245.5990
epoch : 0  val_loss : 83.7334
new model saved at epoch 0 with val_loss 83.73338317871094
epoch : 0  val_loss : 89.9347
------------------------------------------------------------------------------------------------------------------------
epoch : 1   step : 0 train_loss :  197.8448
epoch : 1   step : 10 train_loss :  200.1635
epoch : 1   step : 20 train_loss :  229.1023
epoch : 1  val_loss : 88.2541
epoch : 1  val_loss : 82.6610
new model saved at epoch 1 with val_loss 82.66101837158203
------------------------------------------------------------------------------------------------------------------------
epoch : 2   step : 0 train_loss :  188.0148
epoch : 2   step : 10 train_loss :  216.3756
epoch : 2   step : 20 train_loss :  156.63

epoch : 23   step : 10 train_loss :  61.3209
epoch : 23   step : 20 train_loss :  36.2653
epoch : 23  val_loss : 2.2537
new model saved at epoch 23 with val_loss 2.2536532878875732
epoch : 23  val_loss : 1.8277
new model saved at epoch 23 with val_loss 1.8277130126953125
------------------------------------------------------------------------------------------------------------------------
epoch : 24   step : 0 train_loss :  52.0057
epoch : 24   step : 10 train_loss :  45.7360
epoch : 24   step : 20 train_loss :  31.1103
epoch : 24  val_loss : 1.4231
new model saved at epoch 24 with val_loss 1.4230583906173706
epoch : 24  val_loss : 1.3253
new model saved at epoch 24 with val_loss 1.3253310918807983
------------------------------------------------------------------------------------------------------------------------
epoch : 25   step : 0 train_loss :  50.8748
epoch : 25   step : 10 train_loss :  40.0238
epoch : 25   step : 20 train_loss :  43.3733
epoch : 25  val_loss : 1.3902
epoch 